第一个因子假设：SRMI10日均线上穿30日均线并且成交量增加为买入信号，10日均线下穿30日均线并且成交量增加为卖出信号，用周期为4H的EOSUSDT:binance做通用性检验。

In [1]:
import warnings

warnings.filterwarnings('ignore')
from jaqs_fxdayu.data.dataservice import LocalDataService
from time import time
ds = LocalDataService()
start = time()
path = r'.\min_data\VnTrader_1Min_Db'
props = {'fields': 'open,high,low,close,volume',
         'symbol': 'EOSUSDT:binance',
         'freq': '1Min,5Min,15Min,30Min,1H,4H,6H,8H,12H',
         'start_date':20180601000000}

eos = ds.bar_reader(path,props) #读取数据
print(time()-start)


6.246357202529907


In [2]:
from jaqs_fxdayu.data.hf_dataview import HFDataView

dv2 = HFDataView()
dv2.create_init_dv(eos['4H'].dropna().set_index(["trade_date","symbol"]))

SRMI2 = dv2.add_formula('SRMI_J', "(close-Delay(close,10))/Max(close,Delay(close,10))" ,add_data=True)

Initialize dataview success.


In [3]:
def TimingSignal(td, dv, long='long', short='short',closeLong='closeLong', closeShort='closeShort'\
                 , mhp=None,sl=None,sp=None):
    # step 1：实例化TimingDigger 通过output_folder和output_format指定测试报告的输出路径和输出格式，通过signal_name指定绩效文件名称

    #多空信号分别计算一遍 输出汇总结果
    td.process_signal(
        enter_signal=dv.get_ts(long),
        exit_signal=dv.get_ts(closeLong),
        sig_type="long",  # 信号类型 long/short
        price=dv.get_ts('close'),
        max_holding_period=mhp,  # 最大持有天数 可为空
        stoploss=-sl,  # 止损百分比 负数 可为空
        stopprofit=sp,  # 止盈百分比 正数 可为空
    )

    td.process_signal(
        enter_signal=dv.get_ts(short),
        exit_signal=dv.get_ts(closeShort),
        sig_type="short",  # 信号类型 long/short
        price=dv.get_ts("close"),
        max_holding_period=mhp,  # 最大持有天数 可为空
        stoploss=-sl,  # 止损百分比 负数 可为空
        stopprofit=sp,  # 止盈百分比 正数 可为空
    )

In [4]:
# 进场信号
long = dv2.add_formula("longSRMI","If((Ts_Mean(SRMI_J,10)>Ts_Mean(SRMI_J,30)) && (Delay(Ts_Mean(SRMI_J,10),1)<=Delay(Ts_Mean(SRMI_J,30),1)) && Return(volume,1)>0,2,0)", add_data=True)
short = dv2.add_formula("shortSRMI","If((Ts_Mean(SRMI_J,10)<Ts_Mean(SRMI_J,30)) && (Delay(Ts_Mean(SRMI_J,10),1)>=Delay(Ts_Mean(SRMI_J,30),1)) && Return(volume,1)>0,-2,0)", add_data=True)

# 出场信号
close_long = dv2.add_formula("closeLongSRMI","If(shortSRMI==-2,1,0)", add_data=True)
close_short = dv2.add_formula("closeShortSRMI","If(longSRMI==2,-1,0)", add_data=True)

In [5]:
from jaqs_fxdayu.research import TimingDigger
tdSRMI2 = TimingDigger(output_folder=".", output_format='pdf',signal_name='SRMI')
TimingSignal(tdSRMI2, dv2,'longSRMI','shortSRMI','closeLongSRMI','closeShortSRMI', 100, 0.05, 0.3)

Nan Data Count (should be zero) : 0;  Percentage of effective data: 3%
Nan Data Count (should be zero) : 0;  Percentage of effective data: 3%


In [6]:
tdSRMI2.create_event_report(sig_type="long")

*****-Summary-*****
Event Analysis
                      win    loss     all
t-stat              4.764  -6.122   0.445
p-value             0.003   0.000   0.662
mean                0.100  -0.054   0.009
std                 0.051   0.026   0.085
info_ratio          1.945  -2.041   0.111
skewness           -0.433   0.099   0.569
kurtosis           -1.163  -0.769  -1.139
pct5                0.025  -0.090  -0.081
pct25               0.062  -0.067  -0.061
pct50               0.130  -0.059  -0.020
pct75               0.133  -0.032   0.068
pct95               0.157  -0.015   0.141
occurance           7.000  10.000  17.000
win_ratio             NaN     NaN   0.412
win_mean/loss_mean    NaN     NaN   1.855


In [7]:
tdSRMI2.create_event_report(sig_type="short")

*****-Summary-*****
Event Analysis
                       win   loss     all
t-stat               4.313 -4.290   1.979
p-value              0.002  0.008   0.065
mean                 0.104 -0.053   0.049
std                  0.077  0.028   0.099
info_ratio           1.364 -1.919   0.495
skewness             0.726  0.387   0.522
kurtosis            -0.629 -0.060  -0.566
pct5                 0.015 -0.088  -0.068
pct25                0.052 -0.060  -0.049
pct50                0.082 -0.055   0.037
pct75                0.140 -0.050   0.097
pct95                0.243 -0.014   0.238
occurance           11.000  6.000  17.000
win_ratio              NaN    NaN   0.647
win_mean/loss_mean     NaN    NaN   1.965


In [8]:
tdSRMI2.create_event_report(sig_type="long_short")

*****-Summary-*****
Event Analysis
                       win    loss     all
t-stat               6.232  -7.718   1.778
p-value              0.000   0.000   0.085
mean                 0.103  -0.054   0.029
std                  0.068   0.027   0.094
info_ratio           1.511  -1.993   0.310
skewness             0.597   0.219   0.599
kurtosis            -0.342  -0.456  -0.585
pct5                 0.012  -0.098  -0.083
pct25                0.058  -0.065  -0.057
pct50                0.090  -0.058   0.012
pct75                0.134  -0.043   0.093
pct95                0.237  -0.009   0.190
occurance           18.000  16.000  34.000
win_ratio              NaN     NaN   0.529
win_mean/loss_mean     NaN     NaN   1.916


第二个因子假设：ROC6因子10日均线上穿20日均线并且成交量增加为买入信号，10日均线下穿20日均线并且成交量增加为卖出信号，用周期为4H的EOSUSDT:binance做通用性检验，对参数做了部分修改，改为10日均线上穿30日均线并且成交量增加为买入信号，10日均线下穿30日均线并且成交量增加为卖出信号。

In [9]:
ROC62 = dv2.add_formula('ROC6', '((close/Delay(close, 6))-1)*100', add_data=True)

In [10]:
# 进场信号
long = dv2.add_formula("longROC6","If((Ts_Mean(ROC6,10)>Ts_Mean(ROC6,30)) && (Delay(Ts_Mean(ROC6,10),1)<=Delay(Ts_Mean(ROC6,30),1)) && Return(volume,1)>0,2,0)", add_data=True)
short = dv2.add_formula("shortROC6","If((Ts_Mean(ROC6,10)<Ts_Mean(ROC6,30)) && (Delay(Ts_Mean(ROC6,10),1)>=Delay(Ts_Mean(ROC6,30),1)) && Return(volume,1)>0,-2,0)", add_data=True)

# 出场信号
close_long = dv2.add_formula("closeLongROC6","If(shortROC6==-2,1,0)", add_data=True)
close_short = dv2.add_formula("closeShortROC6","If(longROC6==2,-1,0)", add_data=True)

In [11]:
tdROC6_2 = TimingDigger(output_folder=".", output_format='pdf',signal_name='ROC6')
TimingSignal(tdROC6_2, dv2,'longROC6','shortROC6','closeLongROC6','closeShortROC6', 100, 0.05, 0.3)

Nan Data Count (should be zero) : 0;  Percentage of effective data: 3%
Nan Data Count (should be zero) : 0;  Percentage of effective data: 3%


In [12]:
tdROC6_2.create_event_report(sig_type="long")

*****-Summary-*****
Event Analysis
                      win    loss     all
t-stat              2.863  -7.522   0.239
p-value             0.021   0.000   0.813
mean                0.093  -0.052   0.005
std                 0.092   0.025   0.093
info_ratio          1.012  -2.086   0.051
skewness            1.358   0.712   1.713
kurtosis            1.053  -0.563   2.952
pct5                0.003  -0.079  -0.073
pct25               0.035  -0.067  -0.060
pct50               0.057  -0.060  -0.025
pct75               0.111  -0.035   0.044
pct95               0.251  -0.006   0.151
occurance           9.000  14.000  23.000
win_ratio             NaN     NaN   0.391
win_mean/loss_mean    NaN     NaN   1.789


In [13]:
tdROC6_2.create_event_report(sig_type="short")

*****-Summary-*****
Event Analysis
                       win    loss     all
t-stat               3.536  -6.244   1.624
p-value              0.004   0.000   0.119
mean                 0.098  -0.046   0.035
std                  0.096   0.022   0.102
info_ratio           1.021  -2.081   0.346
skewness             0.923   0.738   1.204
kurtosis            -0.378  -1.056   0.614
pct5                 0.005  -0.066  -0.063
pct25                0.015  -0.062  -0.049
pct50                0.086  -0.056   0.006
pct75                0.116  -0.029   0.089
pct95                0.269  -0.009   0.242
occurance           13.000  10.000  23.000
win_ratio              NaN     NaN   0.565
win_mean/loss_mean     NaN     NaN   2.140


In [14]:
tdROC6_2.create_event_report(sig_type="long_short")

*****-Summary-*****
Event Analysis
                       win    loss     all
t-stat               4.660  -9.888   1.359
p-value              0.000   0.000   0.181
mean                 0.096  -0.049   0.020
std                  0.094   0.024   0.099
info_ratio           1.017  -2.062   0.203
skewness             1.091   0.658   1.418
kurtosis             0.145  -0.719   1.500
pct5                 0.005  -0.072  -0.069
pct25                0.015  -0.063  -0.060
pct50                0.074  -0.060  -0.004
pct75                0.115  -0.026   0.061
pct95                0.303  -0.006   0.237
occurance           22.000  24.000  46.000
win_ratio              NaN     NaN   0.478
win_mean/loss_mean     NaN     NaN   1.940
